In [30]:
import carla
import os
import numpy as np
import cv2
import math
from gym.spaces import Dict, Discrete, Box
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.policies import ActorCriticPolicy
import networkx as nx
import random
from stable_baselines3.common.env_checker import check_env
import time
import torch.nn as nn
import torchvision.models as model
import torch as th
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.monitor import Monitor

class CarEnv(gym.Env):
    def __init__(self):
        super(CarEnv, self).__init__()

        self.client = carla.Client('localhost', 2000)
        self.client.set_timeout(10.0)
        self.world = self.client.load_world('Town01')
        self.blueprint_library = self.world.get_blueprint_library()
        self.image = Box(low=0, high=255, shape=(224, 224, 3), dtype=np.uint8)
        self.status = Box(low=np.array([0.0, -1.0],dtype=np.float32), high=np.array([1.0, 1.0],dtype=np.float32))
        self.observation_space = Dict({"image": self.image, "status": self.status})
        self.action_space = Box(low=-1.0, high=1.0, shape=(2,), dtype=np.float32)
        self.spawn_points = self.world.get_map().get_spawn_points()
        self._map = self.world.get_map()
        self.positive_points = []
        self.actor_list = []
        for i in range(len(self.spawn_points)):
            if self._map.get_waypoint(self.spawn_points[i].location).lane_id > 0:
                self.positive_points.append(self.spawn_points[i])
        self.topology = self._map.get_topology()
        
        self.waypoints_info = {}
        self.G = nx.DiGraph()  
        for i in range(len(self.topology)):
            waypoint1 = self.topology[i][0]
            waypoint2 = self.topology[i][1]
            waypoint1_info = "%s-%s-%s" % (waypoint1.road_id, waypoint1.section_id, waypoint1.lane_id)
            waypoint2_info = "%s-%s-%s" % (waypoint2.road_id, waypoint2.section_id, waypoint2.lane_id)
            distance = math.sqrt(math.pow((waypoint2.transform.location.x - waypoint1.transform.location.x), 2) +
                                         math.pow((waypoint2.transform.location.y - waypoint1.transform.location.y), 2))

            self.waypoints_info[waypoint1_info] = waypoint1
            self.G.add_weighted_edges_from([(waypoint1_info, waypoint2_info, distance)])
        
        start, end = self.generate_route()
        self.path = self.generate_route_info(start, end)
        
        
        self.collision_hist = []
        self.lane_hist = []
        
        self.vehicle_bp = self.blueprint_library.find('vehicle.audi.tt')
        self.vehicle = self.world.spawn_actor(self.vehicle_bp, start)
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, steer=0.0, brake=0.0))
        self.actor_list.append(self.vehicle)

        self.col_bp = self.blueprint_library.find('sensor.other.collision')
        self.col_location = carla.Location(0, 0, 0)
        self.col_transform = carla.Transform(self.col_location)
        self.colsensor = self.world.spawn_actor(self.col_bp, self.col_transform, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision(event))

        self.cam_bp = self.blueprint_library.find('sensor.camera.rgb')
        self.cam_bp.set_attribute("image_size_x", str(224))
        self.cam_bp.set_attribute("image_size_y", str(224))
        self.cam_bp.set_attribute("fov", str(105))
        self.cam_location = carla.Location(0.8, 0, 1.7)
        self.cam_transform = carla.Transform(self.cam_location)
        self.cam_rgb = self.world.spawn_actor(self.cam_bp, self.cam_transform, attach_to=self.vehicle,
                                              attachment_type=carla.AttachmentType.Rigid)
        self.actor_list.append(self.cam_rgb)
#         if not os.path.exists(os.path.join(os.getcwd(), 'Data/RGB')):
#             os.makedirs(os.path.join(os.getcwd(), 'Data/RGB'))
        self.cam_rgb.listen(lambda image: self.img_process(image))
    
        self.lane = None
        self.lane_bp = self.world.get_blueprint_library().find('sensor.other.lane_invasion')
        self.lane_location = carla.Location(0, 0, 0)
        self.lane_transform = carla.Transform(self.lane_location)
        self.lane_sensor = self.world.spawn_actor(self.lane_bp, self.lane_transform, attach_to=self.vehicle,
                                 attachment_type=carla.AttachmentType.Rigid)
        self.lane_sensor.listen(lambda lane: self.lane_callback(lane))
        self.actor_list.append(self.lane_sensor)
        
        self.cur_loc = self.vehicle.get_transform().location
        self.tar_waypoint = self.waypoints_info[self.path[1]]
        self.tar_dis = self.tar_waypoint.transform.location.distance(self.cur_loc)
 #       self.n = 0
        spectator = self.world.get_spectator()
        spectator.set_transform(carla.Transform(self.vehicle.get_transform().location + carla.Location(x=0,y=0,z=2),
                                            self.vehicle.get_transform().rotation))
        
#         spectator = self.world.get_spectator()
#         spectator.set_transform(self.vehicle.get_transform())
#         cur_waypoint = self._map.get_waypoint(self.cur_loc, project_to_road=False)
#         self.world.debug.draw_string(start.location, 'Start', 
#                                      draw_shadow=False, color=carla.Color(r=0, g=255, b=0), life_time=100.0)
#         self.world.debug.draw_string(self.cur_loc, 'Current' ,
#                                      draw_shadow=False, color=carla.Color(r=255, g=0, b=0), life_time=100.0)
#         self.world.debug.draw_string(self.waypoints_info[self.path[0]].transform.location, '%s-%s-%s' 
#                                      % (self.waypoints_info[self.path[0]].road_id,self.waypoints_info[self.path[0]].section_id, 
#                                         self.waypoints_info[self.path[0]].lane_id), 
#                                      draw_shadow=False, color=carla.Color(r=0, g=0, b=255), life_time=100.0)
#         self.world.debug.draw_string(self.waypoints_info[self.path[1]].transform.location, '%s-%s-%s' 
#                                      % (self.waypoints_info[self.path[1]].road_id,self.waypoints_info[self.path[1]].section_id, 
#                                         self.waypoints_info[self.path[1]].lane_id), 
#                                      draw_shadow=False, color=carla.Color(r=0, g=0, b=255), life_time=100.0)
        
    def img_process(self,image):
        """

        :param image:
        """
        data = np.array(image.raw_data)
        data = data.reshape((224, 224, 4))
        data = data[:, :, :3]
        self.front_camera = data
        return data/255.0

    def collision(self, event):
        self.collision_hist.append(event)

    def lane_callback(self, lane):
        lane_types = set(x.type for x in lane.crossed_lane_markings)
        self.lane = eval(['%r' % str(x).split()[-1] for x in lane_types][0])
        self.lane_hist.append(self.lane)
    
    def generate_route_info(self, start, end):
        start_waypoint = self._map.get_waypoint(start.location)
        start_info = "%s-%s-%s" % (start_waypoint.road_id, start_waypoint.section_id, start_waypoint.lane_id)
        end_waypoint = self._map.get_waypoint(end.location)
        end_info = "%s-%s-%s" % (end_waypoint.road_id, end_waypoint.section_id, end_waypoint.lane_id)
        return nx.astar_path(self.G, start_info, end_info)
    
    def generate_route(self):
        start = self.positive_points[random.sample(range(len(self.positive_points)),2)[0]]
        end = self.positive_points[random.sample(range(len(self.positive_points)),2)[1]]
        start_waypoint = self._map.get_waypoint(start.location)
        start_info = "%s-%s-%s" % (start_waypoint.road_id, start_waypoint.section_id, start_waypoint.lane_id)
        end_waypoint = self._map.get_waypoint(end.location)
        end_info = "%s-%s-%s" % (end_waypoint.road_id, end_waypoint.section_id, end_waypoint.lane_id)
        while start_info == end_info:
            start = self.positive_points[random.sample(range(len(self.positive_points)),2)[0]]
            end = self.positive_points[random.sample(range(len(self.positive_points)),2)[1]]
            start_waypoint = self._map.get_waypoint(start.location)
            start_info = "%s-%s-%s" % (start_waypoint.road_id, start_waypoint.section_id, start_waypoint.lane_id)
            end_waypoint = self._map.get_waypoint(end.location)
            end_info = "%s-%s-%s" % (end_waypoint.road_id, end_waypoint.section_id, end_waypoint.lane_id)

        return start, end
            
    def reset(self):
#        self.n = 0
        
        self.world = self.client.reload_world(reset_settings = False)
        
        self.actor_list = []
        self.collision_hist = []
        self.lane_hist = []
        
        start, end = self.generate_route()
        self.path = self.generate_route_info(start, end)
        
        self.vehicle = self.world.spawn_actor(self.vehicle_bp, start)
        self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, steer=0.0, brake=0.0))
        self.actor_list.append(self.vehicle)
        
        self.colsensor = self.world.spawn_actor(self.col_bp, self.col_transform, attach_to=self.vehicle)
        self.actor_list.append(self.colsensor)
        self.colsensor.listen(lambda event: self.collision(event))
        
        self.cam_rgb = self.world.spawn_actor(self.cam_bp, self.cam_transform, attach_to=self.vehicle,
                                              attachment_type=carla.AttachmentType.Rigid)
        self.actor_list.append(self.cam_rgb)
        
        self.cam_rgb.listen(lambda image: self.img_process(image))
        
        self.lane = None
        self.lane_sensor = self.world.spawn_actor(self.lane_bp, self.lane_transform, attach_to=self.vehicle,
                                 attachment_type=carla.AttachmentType.Rigid)
        self.lane_sensor.listen(lambda lane: self.lane_callback(lane))
        self.actor_list.append(self.lane_sensor)
        
        self.cur_loc = self.vehicle.get_transform().location
        self.tar_waypoint = self.waypoints_info[self.path[1]]
        self.tar_dis = self.tar_waypoint.transform.location.distance(self.cur_loc)
        
        spectator = self.world.get_spectator()
        spectator.set_transform(carla.Transform(self.vehicle.get_transform().location + carla.Location(x=0,y=0,z=2),
                                    self.vehicle.get_transform().rotation))
             
        v = self.vehicle.get_velocity()
        speed = int(3.6 * math.sqrt(v.x ** 2 + v.y ** 2 + v.z ** 2))
        a = self.vehicle.get_transform().rotation.yaw
        
        return {'image':self.front_camera,'status':np.array([0.01*speed,a/180]).astype('float32')}


    def step(self, action):
        
        done = False
        last_tar = self.tar_waypoint
        cur_waypoint = self._map.get_waypoint(self.cur_loc, project_to_road=False)
        road_id, section_id, lane_id = cur_waypoint.road_id, cur_waypoint.section_id, cur_waypoint.lane_id 
        if "%s-%s-%s" % (road_id, section_id, lane_id) in self.path:
            self.tar_waypoint = self.waypoints_info[self.path[self.path.index("%s-%s-%s" % (road_id, section_id, lane_id)) + 1]]
        else:
            self.tarwaypoint = last_tar
            
        prev_distance_to_tar = self.tar_dis
                   
        w1, w2, w3, w4 = 1, 0.005, 0.1, 0.01
        speed_limit = 50
        if action[0] >= 0:
            self.vehicle.apply_control(carla.VehicleControl(throttle = float(action[0]), steer = float(action[1]), brake=0.0))
            
        else:
            self.vehicle.apply_control(carla.VehicleControl(throttle=0.0, steer =float(action[1]), brake= float(action[0])))
        self.cur_loc = self.vehicle.get_location()
        self.tar_dis = self.tar_waypoint.transform.location.distance(self.cur_loc)
        advance = prev_distance_to_tar - self.tar_dis 
        
        v = self.vehicle.get_velocity()
        speed = int(3.6 * math.sqrt(v.x ** 2 + v.y ** 2 + v.z ** 2))
        
        speed_diff = speed_limit - speed
        
        if self._map.get_waypoint(self.cur_loc, project_to_road=True, lane_type= carla.LaneType.Any) == None:
            done = True
        else:
            closest_waypoint = self._map.get_waypoint(self.cur_loc, project_to_road=True, lane_type= carla.LaneType.Any)
            distance_to_center = closest_waypoint.transform.location.distance(self.cur_loc)
        
        a = self.vehicle.get_transform().rotation.yaw
        a_tar = closest_waypoint.transform.rotation.yaw
        a_diff = min(abs(a - a_tar), 360 - abs(a - a_tar))
        
        if speed_diff >=0:
            reward = w1*advance + 2*w2*(speed_limit - abs(speed_diff)) + w3*(1-distance_to_center) + w4*(15-a_diff)
            
        else:
            reward = w1*advance + w2*(speed_limit - abs(speed_diff)) + w3*(1-distance_to_center) + w4*(15-a_diff)

        if len(self.collision_hist) != 0:
            done = True
            reward -= 300
            
        if self.waypoints_info[self.path[-1]].transform.location.distance(self.cur_loc) < 1:
            done = True
            reward += 1000
            
        if self.lane == 'Broken':
            reward -= 25 
            
        if len(self.lane_hist) == 2:
            reward -= 50
            done = True
                
        if self.lane == 'NONE':
            reward -= 100
            done = True
        # info must be a dictionary
        info = {}
#        self.n += 1
#        print(reward,'\t',self.n)
        return {'image':self.front_camera,'status':np.array([0.01*speed,a/180]).astype('float32')}, reward, done, info

In [31]:
class CustomCombinedExtractor(BaseFeaturesExtractor):
    def __init__(self, observation_space: Dict):        
        super(CustomCombinedExtractor, self).__init__(observation_space, features_dim=1)

        weights = model.ResNet18_Weights.DEFAULT
        self.network = nn.Sequential(*list(model.resnet18(weights= weights).children())[:-1])
        self.network1 = nn.Linear(512,2000)
        self.status_process = nn.Linear(2, 128)
        # Must define this attribute and must use underscore
        self._features_dim = 2128
           
    def forward(self, observations) -> th.Tensor:
#        print(observations['image'].shape)
        x = self.network(th.Tensor(observations['image']))
#        print(x.size())
        x = self.network1(x.squeeze())
#        print(x.size())
        y = self.status_process(th.Tensor(observations['status'])).squeeze()
#        print(y.size())
        return th.cat((x,y),0)

In [33]:
policy_kwargs = dict(
    features_extractor_class=CustomCombinedExtractor,
    activation_fn=nn.ReLU,
    normalize_images=False,
    net_arch=[2128, 1024, dict(pi=[512,256,128], vf=[256,64])],
)
#eval_env = Monitor(CarEnv())
env = Monitor(CarEnv())
#num_cpu = 2 # Number of processes to use
#env= make_vec_env(lambda:env, n_envs=num_cpu,vec_env_cls=SubprocVecEnv)

model = PPO("MultiInputPolicy",
            env,
            policy_kwargs=policy_kwargs,
            n_steps=4,
            batch_size=4,
            verbose=1,
            gamma = 0.99,
            vf_coef = 0.5,
            gae_lambda = 0.95,
            learning_rate = 0.0002,
            clip_range = 0.2,
            device = 'cuda'
           )

Using cuda device
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


In [25]:
#eval_env = Monitor(CarEnv())
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=5, deterministic = False)
print(f"mean_reward:{mean_reward:.4f} +/- {std_reward:.4f}")

IndexError: invalid index to scalar variable.

In [34]:
model.learn(total_timesteps=200)

IndexError: invalid index to scalar variable.

In [33]:
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=5, deterministic = True)
print(f"mean_reward:{mean_reward:.4f} +/- {std_reward:.4f}")

-43.5860018 	 2.853993863819641


In [24]:
mean_reward

-93.4587904

In [25]:
std_reward

0.9571788366236669

In [28]:
env = CarEnv()

In [9]:
check_env(env)

In [148]:
env.reset()

{'image': array([[[115, 110, 110],
         [115, 110, 110],
         [114, 110, 110],
         ...,
         [125, 121, 121],
         [125, 121, 121],
         [126, 121, 121]],
 
        [[125, 121, 121],
         [126, 121, 121],
         [126, 121, 121],
         ...,
         [120, 115, 115],
         [119, 114, 115],
         [119, 115, 114]],
 
        [[119, 115, 115],
         [120, 117, 117],
         [120, 117, 117],
         ...,
         [120, 117, 117],
         [120, 117, 117],
         [120, 117, 115]],
 
        ...,
 
        [[ 99,  98, 102],
         [ 99,  99, 104],
         [ 99, 101, 104],
         ...,
         [ 46,  46,  50],
         [ 42,  42,  50],
         [ 42,  42,  46]],
 
        [[ 42,  42,  50],
         [ 38,  42,  46],
         [ 38,  38,  46],
         ...,
         [ 95,  90,  90],
         [ 96,  90,  90],
         [ 96,  91,  91]],
 
        [[ 96,  91,  91],
         [ 96,  91,  91],
         [ 96,  91,  91],
         ...,
         [ 71,  69,

In [153]:
env._map.get_waypoint(env.cur_loc, project_to_road=False).lane_id

1

In [29]:
action = np.array([0.5,0.5]).astype('float32')
env.step(action)

({'image': array([[[121, 117, 117],
          [121, 117, 117],
          [121, 117, 117],
          ...,
          [ 13,  22,  13],
          [ 13,  13,  13],
          [ 13,  22,  13]],
  
         [[121, 117, 117],
          [121, 118, 118],
          [122, 118, 118],
          ...,
          [ 13,  22,  13],
          [  0,  13,   0],
          [ 13,  22,  13]],
  
         [[121, 118, 118],
          [122, 118, 118],
          [122, 118, 118],
          ...,
          [ 13,  22,  13],
          [ 13,  22,  13],
          [ 13,  22,  13]],
  
         ...,
  
         [[ 99,  93,  93],
          [ 96,  91,  91],
          [ 59,  53,  53],
          ...,
          [ 73,  69,  69],
          [ 75,  71,  71],
          [ 75,  71,  71]],
  
         [[ 90,  86,  86],
          [ 53,  50,  50],
          [  0,   0,   0],
          ...,
          [ 34,  28,  28],
          [ 73,  69,  69],
          [ 75,  71,  71]],
  
         [[ 38,  34,  34],
          [ 13,   0,  13],
          [  0,

In [154]:
client = carla.Client('localhost', 2000)
# Set timeout to limit the networking operations so that the these don't block the client forever.
# An error will be returned if connection fails.
client.set_timeout(10.0)
# Available maps are Town 01,02,03,04,05,06,07,10
#    world = client.get_world()
print(client.get_available_maps())
#    world = client.get_world()
world = client.load_world('Town01')

['/Game/Carla/Maps/Town01', '/Game/Carla/Maps/Town01_Opt', '/Game/Carla/Maps/Town02', '/Game/Carla/Maps/Town02_Opt', '/Game/Carla/Maps/Town03', '/Game/Carla/Maps/Town03_Opt', '/Game/Carla/Maps/Town04', '/Game/Carla/Maps/Town04_Opt', '/Game/Carla/Maps/Town05', '/Game/Carla/Maps/Town05_Opt', '/Game/Carla/Maps/Town10HD', '/Game/Carla/Maps/Town10HD_Opt']


In [155]:
for i in range(len(env.path)):
    _waypoint = env.waypoints_info[env.path[i]]
    world.debug.draw_string(_waypoint.transform.location, '%s-%s-%s' % (_waypoint.road_id,_waypoint.section_id, _waypoint.lane_id), 
                            draw_shadow=False, color=carla.Color(r=255, g=0, b=0), life_time=100.0)
    time.sleep(3)
    
world.debug.draw_string(env._map.get_waypoint(env.cur_loc, project_to_road=False).transform.location, 
                        '%s-%s-%s' % (env._map.get_waypoint(env.cur_loc, project_to_road=False).road_id,
                                      env._map.get_waypoint(env.cur_loc, project_to_road=False).section_id,
                                      env._map.get_waypoint(env.cur_loc, project_to_road=False).lane_id), draw_shadow=False,
                                    color=carla.Color(r=255, g=0, b=0), life_time=100.0)

time.sleep(5)

In [3]:
import torchvision.models as models

In [6]:
import torch

In [8]:
torch.__version__

'1.12.0'

In [4]:
model = models.resnet18()

In [5]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  